# 🚀 How to automate Excel using Python

![header](0-Images/header.png "header")

> In this workshop, I will be sharing my favorite ways to **automate Microsoft Excel using Python**. In particular, we will be using the open-source tools: `Pandas`, `xlwings` & `plotly`. After this workshop, you will be able to create a custom Python script that allows you to combine excel files & create charts out of them.

### 👉 Find this Jupyter Notebook & all examples files here: https://pythonandvba.com/workshop

***

# 1. About Me 👱‍♂️

Hi 👋, I'm Sven - A data analyst from Germany, currently living in Tokyo <br>

👱‍♂️ 32 years old <br>
📊 I have been working as a business/data analyst at a German engineering & technology company for over 7 years <br>
📚 Le Wagon Tokyo alumni - 9 weeks full-time Data Science Boot Camp [2020] <br>
📝 Languages: `Python` and `VBA` <br><br>
📺 I have a small YouTube channel: https://youtube.com/c/codingisfun <br>
On my YouTube channel I am creating tutorials for people who:
- … are looking for a way to leverage the power of Python in their day-to-day work.
- … are struggling with repetitive tasks in Excel and are looking for a way to use Python and VBA.
- … want to learn Data Analysis & Data Science to perform meaningful and impactful analyses.
- … are working with Excel and found themselves thinking - "there has to be a better way."

---
🛑 **DISCLAIMER** 🛑
> **I am not a software developer**, but I am passionate about finding ways to use `Python` and `VBA` to be more efficient and effective in business settings

# 2. Before we start ... 🤓

## Prerequisite:

> Basic understanding of Python and Excel

## Why bother using Microsoft Excel? Use ____ (a database/jupyter notebook/pandas/...)!

- **Over 750 million** MS Excel users worldwide (source: https://www.investintech.com/resources/blog/archives/5718-experts-predict-the-future-of-excel-in-business-intelligence.html)
- MS Excel is an **integral part of most business organizations** across the world (even after three decades!)
- **Multipurpose**: generating memos, track sales trends, handling other business data, ...
- MS Excel has a **simple interface**, that allows users to easily understand it and also perform basic activities
- There is a **huge amount of help** out there for Excel users
- Big systems are great for delivering clean data – Excel allows you to do something with it (can be an issue for automation!)

#### **What is the most used feature in any business intelligence solution?**

> **It is the `Export to Excel` button.**

# 3. Excel Automation Examples 🤖

## Install Dependencies

In [ ]:
#!pip install pandas --quiet
#!pip install openpyxl --quiet
#!pip install plotly-express --quiet
#!pip install xlwings --quiet

## 3.1 Split Column Values

### 📢 Purpose:<br><br> Separate Excel Data into Workbooks by Column Values

In [ ]:
from pathlib import Path
import pandas as pd  # pip install pandas

In [ ]:
# Locate examples files & create output directory
EXCEL_FILE_PATH = Path.cwd() / "1-Split-Colmun-Values"/ "Financial_Data.xlsx"
OUTPUT_DIR = Path.cwd() / "1-Split-Colmun-Values"/ "OUTPUT"

if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir()

In [ ]:
# Load Dataframe
df = pd.read_excel(EXCEL_FILE_PATH)
df.head()

In [ ]:
# Get unique values from any particular column
column_name = "Country"
unique_values = df[column_name].unique()
unique_values

In [ ]:
# Query/Filter dataframe, example:
df.query("Country=='Japan'")

In [ ]:
# Query/Filter the dataframe and export the filtered dataframe as an Excel file
for unique_value in unique_values:
    df_output = df.query(f"{column_name} == @unique_value")
    df_output = df_output.query("Year==2021")
    output_path = OUTPUT_DIR / f"{unique_value}_2021.xlsx"
    df_output.to_excel(output_path, sheet_name=unique_value, index=False)

### 🚨 Some Remarks

#### No.1

Pandas uses `openpyxl` to manipulate workbooks
> One of the main advantage is, that `OpenPyXL` does not require Excel. That makes it so fast!

As funny as it sounds, but OpenPyXL cannot use a running instance of Excel.
In fact, Excel files are actually a zipped collection of **XML files**. You can easily explore and edit the complete structure of your Excel workbook. All you have to do: Rename the .xlsx file to .zip

#### No.2

`OpenPyXL` (&`Pandas`) will not save/remember the formatting of the respective worksheet (colors, font styling, conditional formatting, ..) <br>

This might be not a big issue when dealing with large amount of data. Yet, it can be a disadvantage if you want to create 'pretty' Excel reports/workbooks

#### No.3

This was a simplified example. In real-life, the dataset might look way messier

## 3.2 Merge Excel Files (Simple)

### 📢 Purpose: <br><br> Merge multiple Excel files, perform calculations & export an interactive chart

In [ ]:
from pathlib import Path
import pandas as pd  # pip install pandas
import plotly.express as px  # pip install plotly-express

In [ ]:
# Locate examples files
INPUT_DIR = Path.cwd() / "2-Merge-Excel-Files-Simple" / "INPUT"
OUTPUT_DIR = Path.cwd() / "2-Merge-Excel-Files-Simple"

In [ ]:
# Create empty dataframe, iterate over input directory, append all excel files to empty dataframe
df = pd.DataFrame()

# If you want to iterate over all folders including subfolders, use: 
# for file in INPUT_DIR.rglob("*"):

for file in INPUT_DIR.iterdir():
    if file.suffix == ".xlsx":
        df = df.append(pd.read_excel(file), ignore_index=True)
    
df['Country'].unique()

In [ ]:
df.head()

In [ ]:
df = df.groupby(by="Country").sum()[[" Sales", "Profit"]]
df

In [ ]:
df["Profit Margin %"] = (df["Profit"] / df[" Sales"]) * 100
df

In [ ]:
# Plot Profit situation & export grouped dataframe to Excel
fig = px.bar(
    df,
    x=df.index,
    y=" Sales",
    color="Profit Margin %",
    color_continuous_scale=["red", "yellow", "green"],
    template="plotly_white",
    title="<b>Profit Analysis</b>",
)
fig.write_html(str(OUTPUT_DIR / "profit_analysis.html"))
df.to_excel(OUTPUT_DIR / "summary.xlsx")
fig

### 🚨 Some Remarks

- Making use of the different plotting libraries, e.g. interactive visualizations using Plotly
- **Simplified Example** | In real-life, more often than not, datasets are not so well-structured

## 3.3 Merge Excel Files (Real Life Example)

### 📢 Purpose: <br><br>Merge multiple Excel files, perform calculations & insert (Excel) charts [keep excel formatting & formulas]

In [ ]:
from pathlib import Path
import pandas as pd  # pip install pandas
import xlwings as xw  # pip install xlwings

In [ ]:
# Locate examples files
INPUT_DIR = Path.cwd() / "3-Merge-Excel-Files-Real-Life" / "INPUT"
OUTPUT_DIR = Path.cwd() / "3-Merge-Excel-Files-Real-Life"

In [ ]:
# Create an empty workbook. This wb will be our final output
summary_wb = xw.Book()
summary_sht = summary_wb.sheets[0]
summary_sht.name = "Summary"

In [ ]:
# Iterate over each file in directory. Copy cell range & paste it into summary workbook (keep formatting & formuals)
for file in INPUT_DIR.iterdir():
    if file.suffix == ".xlsx":
        wb = xw.Book(file)
        wb.app.display_alerts = False
        #wb.app.visible = False
        sht = wb.sheets("Overview")
        values = sht.range("B5").expand().copy()
        dest_last_row = summary_sht.range("A1").expand().last_cell.row + 1
        summary_sht.range(f"A{dest_last_row}").paste()
        wb.close()

In [ ]:
# Insert & style header of our summary workbook
header = [
    "Segment",
    "Country",
    "Product",
    "Discount Band",
    "Units Sold",
    "Sale Price",
    "Gross Sales",
    "Discounts",
    "Sales",
    "COGS",
    "Profit",
]
header_row = summary_sht.range("A1:K1")
header_row.value = header
header_row.font.bold = True
header_row.font.color = (255, 255, 255)
header_row.color = (119, 136, 153)

In [ ]:
# Use pandas to perform calculations, e.g. data aggregation
df = summary_sht.range("A1").options(pd.DataFrame, expand="table").value
data_by_country = df.groupby(by="Country").sum()[["Sales"]]
data_by_country

In [ ]:
# Insert grouped dataframe into our summary workbook
summary_sht.range("M1").value = data_by_country

In [ ]:
# Insert EXCEL chart
chart = summary_sht.charts.add(
    left=summary_sht.range("M8").left, 
    top=summary_sht.range("M8").top,
    width=400,
    height=200,
)
chart.set_source_data(summary_sht.range("M1").expand())
chart.chart_type = "bar_stacked"

In [ ]:
# Insert PANDAS chart
ax = data_by_country.sort_values(by="Sales").plot(kind="barh")
fig = ax.get_figure()
summary_sht.pictures.add(
    fig,
    name="Sales",
    update=True,
    left=summary_sht.range("M23").left,
    top=summary_sht.range("M23").top,
    width=400,
    height=200,
)

In [ ]:
# Clean up workbook, save workbook & quit Excel instance (if it is the only wb open)
summary_sht.autofit()
summary_wb.save(OUTPUT_DIR / "summary.xlsx")
if len(summary_wb.app.books) == 1:
    summary_wb.app.quit()
else:
    summary_wb.close()

### 🚨 Some Remarks

 **`xlwings` ..**
 - .. is just a smart wrapper around `pywin32` on Windows and `appscript` on Mac.
 - .. works nicely together with Pandas
 - .. (Excel) is not ideal when dealing with larger datasets.
 - .. has actually much more to offer. You can, for instance:
     - write user-defined-functions (UDF) in Python
     - run Python directly in Excel

**There are more Python packages available to work with Excel files:** <br>
openpyxl, xlsxwriter, pyxlsb, pylightxl, xlrd, xlwt, xlutils, PyXLL, .. <br>
> Have a look here: https://www.python-excel.org/

## 3.4 Run Python in Excel [OPTIONAL]

# 4. BONUS - A `free` gift for you!🎉

## `Issue`: Creating Pandas Dataframe from 'messy' Excel file

![MessyFile](0-Images/messy-example.jpg "MessyFile")

**Using pd.read_excel() default parameters**

In [ ]:

from pathlib import Path
import pandas as pd

excel_file = Path.cwd() / "5-Bonus" / "Sales_Data.xlsx"
df = pd.read_excel(excel_file)
df

---

**Specifying pd.read_excel() parameters**

In [ ]:
df = pd.read_excel(
    io=excel_file,
    engine='openpyxl',
    sheet_name='Overview',
    skiprows=3,
    usecols='B:L',
    nrows=105,
)
df.head()

## `Solution`: Free Excel Addin *My Tool Belt* 🚀

![MyToolBelt](0-Images/rsz_mytoolbelt.png "MyToolBelt")

The add-in creates the Python code to correctly read in the excel file and construct the dataframe. Instead of messing around with all of the available options in the read_excel method, it does it for you. Particularly handy when the tables are formatted and have merged cells.
 
The add-in is written in `VBA`, and it will translate the cell range into the following pandas arguments:

- io [File Name]
- sheet_name
- skiprows [Number of lines to skip (int) at the start of the file]
- usecols [Excel column letters and column ranges (e.g. “A:E”)]
- nrows [Number of rows to parse]

### 📢 𝗗𝗢𝗪𝗡𝗟𝗢𝗔𝗗 𝗧𝗛𝗘 𝗙𝗥𝗘𝗘 𝗔𝗗𝗗-𝗜𝗡 𝗛𝗘𝗥𝗘:
► https://pythonandvba.com/mytoolbelt

### 📺 𝗙𝗨𝗟𝗟 𝗔𝗗𝗗-𝗜𝗡 𝗧𝗨𝗧𝗢𝗥𝗜𝗔𝗟:
► https://youtu.be/PmJ9rkKGqrI

# 5. My take on Excel automation in general & VBA vs. Python ⚡

## Should you try to automate all Excel files?

![meme](0-Images/meme-automation1.jpg "Meme")

---

### Use the right tool for the right job:
### 👉 MS Excel, PowerQuery, PowerBI, Database, VBA, Python, ...

![tool](0-Images/right-tool.jpeg "Right Tool")

## VBA or Python? Which one should I use?

**Overall:** <br>
> Computer programming languages are like **tools in a toolbox**. Different tools are designed to solve different problems.
- Python is a General-Purpose Language
- Visual Basic for Applications (VBA) is a simple, but powerful programming language to extend Office applications

**VBA:**<br>
  👍 Syntax is very intuitive. Example: Range("A1:C7").ClearContents <br>
  👍 No additional installation is required (huge advantage!). Built in `Visual Basic Editor (VBE)` <br>
  👎 It is restricted to Office Applications <br>

**Python:**<br>
  👍 Has very powerful packages (for data analysis, machine-/deeplearning, web development, ..) <br>
  👍 Python is one of the fastest-growing programming languages <br>
  👎 Requires additional setup: Python(distribution), Editor (other than Word or Notepad 😅)<br>
 
___
Python might be more powerful than VBA but like I said earlier, what matters is what you intend to achieve. There is no need killing a mosquito with an atomic bomb when a simple spray can do the job.

At times I find VBA-Excel more appropriate for a certain task and much more efficient than Python. But I also like the powerful Python packages.

# 6. Connect with me 💬

> 📺 **YouTube:** https://youtube.com/c/codingisfun <br>
> 🌎 **Website:** https://pythonandvba.com <br>
> 📝 **GitHub:** https://github.com/Sven-Bo <br>
> ⭐ **Discord:** https://pythonandvba.com/discord <br>
> 💬 **EMail:** contact@pythonandvba.com <br>